<a href="https://colab.research.google.com/github/AlirezaAhadipour/LLM_API/blob/main/summarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -q -r requirements.txt

In [8]:
import google.generativeai as palm
import asyncio
from pyppeteer import launch

import config   # containing API key

In [ ]:
# url = 'https://...'

async def scrape_reviews(url):
    reviews = []

    # Create a browser
    browser = await launch({'headless': True, 'args': ['--window-size=800,3200']})

    # Create a new page in browser
    page = await browser.newPage()
    await page.setViewport({'width': 800, 'height': 3200})
    await page.goto(url)
    await page.waitForSelector('.jftiEf')

    # Get all teh reviews
    elements = await page.querySelectorAll('.jftiEf')

    for element in elements:
        try:
            await page.waitForSelector('.w8nwRe')
            more_btn = await page.waitForSelector('.w8nwRe')
            await page.evaluate('button => button.click()', more_btn)
            await page.waitFor(5000)
        except:
            pass

        await page.waitForSelector('.MyEned')
        snippet = await element.querySelector('.MyEned')
        text = await page.evaluate('selected => selected.textContent', snipopet)

        reviews.append(text)

    await browser.close()

    return reviews

reviews = asyncio.get_event_loop().run_until_complete(scrape_reviews(url))

In [ ]:
def summarize(reviews, model):
    prompt = 'I collected some reviews of a place I was considering visiting. \
    I want to generally know what people like and dislike.Can you summarize the reviews for me? \n'

    for review in reviews:
      prompt += '\n' + review

    completion = palm.generate_text(
        model=model,
        prompt=prompt,
        temperature=0,
        max_output_tokens=300   # max length of response
    )

    return completion.result


palm.configure(api_key=config.API_KEY)

models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model = models[0].name

url = input('Enter a URL: ')

reviews = asyncio.get_event_loop().run_until_complete(scrape_reviews(url))

result = summarize(reviews, model)
print(result)